## The dataset

We will be using the NIDS Coronavirus Rapid Mobile (CRAM) Survey Wave 5 2021 dataset. It is comprised of three tables that include 62 features:

`Personal_2021`
This table contains personal information such as, gender, interview month, date of birth, and marital status. 

`Household_2021`
This table provides information on households such as the household income, the province and geographical type of the household, and if the household has access to water and electricity.

`Employment_2021`
This table provides data on individuals, such as whether they are being paid a wage or salary, how many days they have worked in the past month, and if they have not worked, the reason for absence.

*Note that in the 2021 data, only one person was interviewed per household. Therefore, the `pid` in `Personal_2021` is the same as the `pid` in `Household_2021` for each individual. This means there is no separate household ID (`hhid`) in the 2021 dataset—only `pid` is used to represent both personal and household data.*

Below is an entity relationship diagram (ERD) of the `NIDS-CRAM Wave 5 2021` database:

#  

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/2021-database-erd.png"  style="width:40%";/>
</div>

This ERD illustrates the relationships between the tables in the database.

## Loading the database

Before we begin, we need to prepare our SQL environment.

We can do this by loading the magic command `%load_ext sql`.

In [6]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook. 
# If you get an error here, make sure that mysql and pymysql are installed correctly. 

%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [8]:
%sql sqlite:///NiDS_CRAM_2021.db

We can also load our database using a sqlite3 connection for those commands that are not supported by IPython.

In [16]:
import sqlite3
# Connect to the NiDS_CRAM database
conn = sqlite3.connect('NiDS_CRAM_2021.db')

In [12]:
cursor = conn.cursor()



- Retrieve all individuals who belong to households with more than 20 residents.



In [13]:
%%sql
Select *
FROM Household_2021
WHERE nopres > 20;

 * sqlite:///NiDS_CRAM_2021.db
Done.


pid,prov,area,area_des,dwltyp,watsrc,enrgelec,nopres,nou7res,no7to17res,nocld,no60res,hhincchld,incchld,hhincgovpen,incgovpen,hhinccv,incgov,hhinc,hhinc_brac,hhincmain,hhincsrc1,hhincsrc2,hhincsrc3,hhincsrc4,hhfdyn,fdayn,fdaskp,fdcyn,fdcskp
304234,7,2,3,None,Yes,Yes,34,5,10,17,2,10,None,2,None,4,Yes,23000,None,1,1,3,None,None,Yes,Yes,3,Yes,3
312472,5,3,5,None,No,Yes,23,5,7,10,1,5,None,2,None,0,Yes,None,3,7,1,3,7,None,No,No,None,No,None
318567,5,3,1,None,Yes,Yes,22,5,5,11,1,None,Yes,1,None,2,Yes,None,None,3,3,None,None,None,Yes,Yes,3,Yes,None
381310,5,3,5,2,Yes,Yes,21,3,3,14,1,8,None,1,None,1,Yes,None,4,1,1,3,None,None,Yes,No,None,No,None
381349,5,3,1,2,Yes,Yes,31,3,5,22,1,9,None,1,None,0,No,None,4,1,1,3,None,None,No,No,None,No,None
581096,5,3,1,2,Yes,Yes,21,3,8,9,1,8,None,1,None,0,No,7980,None,3,1,3,None,None,Yes,Yes,3,Yes,3
745922,5,3,4,None,Yes,Yes,24,3,8,11,2,None,No,2,None,0,Yes,None,4,3,3,None,None,None,No,No,None,No,None
750692,5,3,6,None,No,Yes,29,8,11,9,1,19,None,1,None,3,Yes,3000,None,3,3,None,None,None,No,No,None,No,None
769086,3,3,2,None,Yes,Yes,22,7,7,7,1,8,None,1,None,0,No,2300,None,3,3,None,None,None,Yes,No,None,No,None
770712,5,3,5,2,Yes,Yes,24,5,6,11,2,7,None,2,None,0,Yes,None,4,7,3,7,None,None,Yes,Yes,3,Yes,3


- Calculate the average number of days worked (`emdays`) by individuals who have reported any income (`eminc`). Ensure that query only includes individuals with non-null and income values greater than 0 from the `Employment_2021` table.



In [15]:
%%sql
SELECT AVG(emdays)
FROM Employment_2021
WHERE eminc >0 AND emdays IS NOT NULL;

 * sqlite:///NiDS_CRAM_2021.db
Done.


AVG(emdays)
4.522093813732155


- Count the number of households where **every member** is at or above retirement age (i.e., the number of people in the household is equal to the number of people aged 60 or older).


In [ ]:
%%sql
SELECT COUNT(*) AS Retirement_Only_Households
FROM Household_2021
WHERE nopres IS NOT NULL  
  AND no60res IS NOT NULL 
  AND nopres = no60res;   

 * sqlite:///NiDS_CRAM_2021.db
Done.


Retirement_Only_Households
147


- Display the `pid`, the total number of children in the household, and the number of child grants.

- Filter the results to only show the rows for the following `pid` values: `311305`, `787796`, `323054`, and `491746`.


In [24]:
%%sql

SELECT 
    pid, 
    COUNT(CASE WHEN no7to17res < 18 THEN 1 END) AS total_children,
    MAX(hhincchld) AS number_of_child_grants
FROM Household_2021
WHERE pid IN (311305, 787796, 323054, 491746)
GROUP BY pid;


 * sqlite:///NiDS_CRAM_2021.db
Done.


pid,total_children,number_of_child_grants
311305,1,0
323054,1,1
491746,1,0
787796,1,2


-  Query that divides the total household income (`hhinc`) by the number of residents in the household (`nopres`) to calculate the income per person. Then, based on the income per person, categorise each household into the poverty lines mentioned above.

- Filter out any rows with missing or null values in the columns and handle any division by zero. 

- Display the percentage of households that fall into each poverty category.




In [26]:
%%sql

WITH income_per_person AS (
    SELECT 
        hhinc, 
        nopres,
        hhinc / NULLIF(nopres, 0) AS income_per_person
    FROM Household_2021
    WHERE hhinc IS NOT NULL AND nopres IS NOT NULL
),
poverty_categories AS (
    SELECT 
        CASE 
            WHEN income_per_person < 624 THEN 'FPL'
            WHEN income_per_person >= 624 AND income_per_person < 890 THEN 'LBPL'
            WHEN income_per_person >= 890 AND income_per_person < 1135 THEN 'UBPL'
            ELSE 'None'
        END AS poverty_line
    FROM income_per_person
)
SELECT 
    poverty_line, 
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM poverty_categories), 2) AS percentage
FROM poverty_categories
GROUP BY poverty_line;


 * sqlite:///NiDS_CRAM_2021.db
Done.


poverty_line,percentage
FPL,50.73
LBPL,11.44
None,30.64
UBPL,7.18


- Calculate the active labour force participation rate as a percentage of the total potential labour force in the Employment_2021 table. Ensure to exclude any rows with NULL values in both the relevant columns.



In [38]:
%%sql
WITH LabourForce AS (
    SELECT 
        em,
        unems
    FROM 
        Employment_2021
    WHERE 
        em IS NOT NULL AND 
        unems IS NOT NULL
)

SELECT 
    COUNT(CASE WHEN em = 1 OR unems = 'Yes' THEN 1 END) * 100.0 / 
    COUNT(*) AS active_labour_force_participation_rate
FROM 
    LabourForce
WHERE 
    em != 3;  -- Exclude retired individuals


 * sqlite:///NiDS_CRAM_2021.db
Done.


active_labour_force_participation_rate
56.82501503307276



* Calculate the total income for males (`gen = 'Male'`) who reported any positive income (`eminc > 0`) and compute the average income by dividing this total by the number of males with positive income.
* Repeat the same calculation for females (`gen = 'Female'`).
* Return the average income for both males and females.
- Exclude rows with NULL values in either column and only include those who are actually earning (`eminc > 0`).



In [41]:
%%sql
SELECT 
    p.gen AS gender,
    SUM(e.eminc) AS total_income,
    COUNT(*) AS number_of_earners,
    AVG(e.eminc) AS average_income
FROM 
    Employment_2021 e
JOIN 
    Personal_2021 p ON e.pid = p.pid
WHERE 
    p.gen IN ('Male', 'Female') AND
    e.eminc IS NOT NULL AND
    e.eminc > 0
GROUP BY 
    p.gen;

 * sqlite:///NiDS_CRAM_2021.db
Done.


gender,total_income,number_of_earners,average_income
Female,4341132,841,5161.86920332937
Male,5697955,635,8973.157480314962


- Calculate the average household size and the corresponding average income per person (`hhinc / nopres`).

- Filter out any households with a missing or zero values in `nopres` or `hhinc`.

Determine the average income per person for:
* Households with more than 8 residents
* Households with 8 or fewer residents


In [44]:
%%sql
SELECT 
    CASE 
        WHEN nopres > 8 THEN 'More than 8 residents'
        ELSE '8 or fewer residents'
    END AS household_size_category,
    AVG(hhinc * 1.0 / nopres) AS average_income_per_person
FROM 
    Household_2021
WHERE 
    nopres IS NOT NULL AND 
    nopres > 0 AND  -- Exclude households with zero residents
    hhinc IS NOT NULL AND 
    hhinc > 0  -- Exclude households with zero income
GROUP BY 
    household_size_category;

 * sqlite:///NiDS_CRAM_2021.db
Done.


household_size_category,average_income_per_person
8 or fewer residents,2049.0232206012383
More than 8 residents,374.0368837627816


- Calculate the average household size (`nopres`) for households that received at least one COVID-19 grant (`hhinccv > 0`).



In [47]:
%%sql
SELECT 
    AVG(nopres) AS average_household_size  -- Calculate the average household size
FROM 
    Household_2021
WHERE 
    hhinccv > 0;  -- Filter for households that received at least one grant

 * sqlite:///NiDS_CRAM_2021.db
Done.


average_household_size
6.370698644421272


- Calculate the total number of households that experienced this issue.


In [48]:
%%sql
SELECT 
    COUNT(*) AS total_households_out_of_money
FROM 
    Household_2021
WHERE 
    hhfdyn = 'Yes';  -- Count households that ran out of money to buy food


 * sqlite:///NiDS_CRAM_2021.db
Done.


total_households_out_of_money
2312


- Calculate the percentage of males (`gen = 'Male'`) and females (`gen = 'Female'`) who had a job in March (em = 1).


In [49]:
%%sql
WITH EmploymentCounts AS (
    SELECT 
        p.gen, 
        COUNT(*) AS total_individuals,
        SUM(CASE WHEN e.em = 1 THEN 1 ELSE 0 END) AS employed_individuals
    FROM 
        Personal_2021 p  -- Table containing the gender information
    JOIN 
        Employment_2021 e ON p.pid = e.pid  -- Assuming 'pid' is the common identifier
    WHERE 
        p.gen IN ('Male', 'Female')  -- Filter for only Male and Female
    GROUP BY 
        p.gen
)

SELECT 
    gen,
    total_individuals,
    employed_individuals,
    (employed_individuals * 100.0 / total_individuals) AS employment_percentage
FROM 
    EmploymentCounts;

 * sqlite:///NiDS_CRAM_2021.db
Done.


gen,total_individuals,employed_individuals,employment_percentage
Female,3614,1089,30.132816823464307
Male,2248,985,43.81672597864769


- Calculate the average household income (`hhinc`) for households with access to electricity (`enrgelec = 'Yes'`) versus those without electricity (`enrgelec = 'No'`). 



In [50]:
%%sql
WITH IncomeAnalysis AS (
    SELECT 
        enrgelec,
        AVG(hhinc) AS average_income
    FROM 
        Household_2021  -- Ensure you use the correct table name
    WHERE 
        enrgelec IN ('Yes', 'No') AND  -- Filter for households with and without electricity
        hhinc IS NOT NULL  -- Exclude rows with NULL household income
    GROUP BY 
        enrgelec
)

SELECT 
    MAX(CASE WHEN enrgelec = 'Yes' THEN average_income END) AS avg_income_with_electricity,
    MAX(CASE WHEN enrgelec = 'No' THEN average_income END) AS avg_income_without_electricity,
    ROUND(
        MAX(CASE WHEN enrgelec = 'Yes' THEN average_income END) - 
        MAX(CASE WHEN enrgelec = 'No' THEN average_income END)
    ) AS income_difference
FROM 
    IncomeAnalysis;

 * sqlite:///NiDS_CRAM_2021.db
Done.


avg_income_with_electricity,avg_income_without_electricity,income_difference
5799.6265807091495,3091.9298245614036,2708.0


- Calculate the percentage of households living in informal houses that have access to piped or tap water.


In [51]:
%%sql
SELECT 
    COUNT(*) AS total_informal_houses,  -- Total number of informal houses
    SUM(CASE WHEN watsrc = 'Yes' THEN 1 ELSE 0 END) AS households_with_water,  -- Count of informal houses with water access
    (SUM(CASE WHEN watsrc = 'Yes' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) AS percentage_with_water  -- Percentage calculation
FROM 
    Household_2021
WHERE 
    dwltyp = 3  -- Focus on informal houses
    AND watsrc IS NOT NULL;  -- Exclude NULL values for water source

 * sqlite:///NiDS_CRAM_2021.db
Done.


total_informal_houses,households_with_water,percentage_with_water
524,346,66.03053435114504
